# 基于CNN的手写数字图像识别应用

In [39]:
import moxing.tensorflow as mox
import os

根据数据存储和数据输出设置data_url和train_url以及log_url（使用tf.flags）

In [40]:
####### your coding place： begin  ###########
# 此处必须修改为用户数据桶位置

#数据在OBS的存储位置。
# eg. s3:// ：统一路径输入
#     /uBucket ：桶名，用户的私有桶的名称 eg. bucket
#     /notebook/data/： 文件路径

data_url = 's3://zhh-obs001/test-modelarts/dataset-mnist/' 

####### your coding place： end  ###########

In [41]:
train_url = './cache/model/'          #训练输出位置。
if not mox.file.exists(data_url):
    raise ValueError('Plese verify your data url!')
if mox.file.exists(train_url):
    mox.file.remove(train_url,recursive=True)
mox.file.make_dirs(train_url)
log_url = './cache/log/'            # 日志存放位置
if mox.file.exists(log_url):
    mox.file.remove(log_url,recursive=True)
mox.file.make_dirs(log_url)

 通过mox 能够将数据拷贝到本地，这样能够加快训练。操作如下：

In [42]:
# 本地创建数据存储文件夹
local_url = './cache/local_data/'
if mox.file.exists(local_url):
    mox.file.remove(local_url,recursive=True)
os.makedirs(local_url)

#将私有桶中的数据拷贝到本地mox.file.copy_parallel（）
"""
  Copy all files in src_url to dst_url. Same usage as `shutil.copytree`.
  Note that this method can only copy a directory. If you want to copy a single file,
  please use `mox.file.copy`

  Example::

    copy_parallel(src_url='/tmp', dst_url='s3://bucket_name/my_data')

  Assuming files in `/tmp` are:

  * /tmp:
      * |- train
          * |- 1.jpg
          * |- 2.jpg
      * |- eval
          * |- 3.jpg
          * |- 4.jpg

  Then files after copy in `s3://bucket_name/my_data` are:

  * s3://bucket_name/my_data:
      * |- train
          * |- 1.jpg
          * |- 2.jpg
      * |- eval
          * |- 3.jpg
          * |- 4.jpg

  Directory `tmp` will not be copied. If `file_list` is `['train/1.jpg', 'eval/4.jpg']`,
  then files after copy in `s3://bucket_name/my_data` are:

  * s3://bucket_name/my_data
      * |- train
          * |- 1.jpg
      * |- eval
          * |- 4.jpg

  :param src_url: Source path or s3 url
  :param dst_url: Destination path or s3 url
  :param file_list: A list of relative path to `src_url` of files need to be copied.
  :param threads: Number of threads or processings in Pool.
  :param is_processing: If True, multiprocessing is used. If False, multithreading is used.
  :param use_queue: Whether use queue to manage downloading list.
  :return: None
"""
mox.file.copy_parallel(data_url, local_url)
data_url = local_url
os.listdir(data_url)

['t10k-images-idx3-ubyte',
 't10k-images-idx3-ubyte.gz',
 't10k-labels-idx1-ubyte',
 't10k-labels-idx1-ubyte.gz',
 'train-labels-idx1-ubyte.gz',
 'train-images-idx3-ubyte',
 'train-images-idx3-ubyte.gz',
 'train-labels-idx1-ubyte']

In [43]:
print(os.getcwd())
all_files = [f for f in os.listdir(os.getcwd())]#输出根path下的所有文件名到一个列表中
#对各个文件进行处理
print(all_files)

/home/ma-user/work
['cache']


In [44]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os

import numpy
from scipy import ndimage

from six.moves import urllib

import tensorflow as tf
tf.reset_default_graph()


**说明 1**  &#160; &#160; 函数 tf.flags.DEFINE_string('data_url', None, 'Dir of dataset')  数据路径。
                  函数tf.flags.DEFINE_string('train_url', None, 'Train Url') 日志以及生产模型的存储路径。 当脚本运行的时候可以利用tf.flags传入参数。

In [45]:
DATA_DIRECTORY = data_url
# Params for MNIST
IMAGE_SIZE = 28
NUM_CHANNELS = 1
PIXEL_DEPTH = 255
NUM_LABELS = 10
VALIDATION_SIZE = 5000  # Size of the validation set.

# filenames = ['train-images-idx3-ubyte.gz','train-labels-idx1-ubyte.gz','t10k-images-idx3-ubyte.gz',
#              't10k-labels-idx1-ubyte.gz']

# for filename in filenames:
#   filepath = os.path.join(data_url, filename)
#   if not mox.file.exists(filepath):
#     raise ValueError('MNIST dataset file %s not found in %s' % (filepath, local_url))



####  &#160;&#160;训练的main函数包含三个部分，输入定义、模型定义和运行。

1） 输入函数：input_fn(run_mode, **kwargs) 用户可以根据自己的输入编写。本例中通过迭代的方式从数据集中取数据。


2） 模型定义：def model_fn(inputs, run_mode, **kwargs): 模型结构定义函数，返回 mox.ModelSpec(），用户作业模式定义返回值。
但需要满足如下条件：

 &#160;&#160; For run_mode == ModeKeys.TRAIN: `loss` is required.
  
  &#160;&#160;  For run_mode == ModeKeys.EVAL: `log_info` is required.
  
  &#160;&#160;  For run_mode == ModeKeys.PREDICT: `output_info` is required.
  
  &#160;&#160;  For run_mode == ModeKeys.EXPORT: `export_spec` is required.
  

3） 执行训练： mox.run(），训练的过程中可指定optimizer的一些设置，训练batch的大小等，设置内容如下：


 &#160;&#160; 输入函数， input_fn: An input_fn defined by user. Allows tfrecord or python data. Returns  input tensor list.
 
 &#160;&#160;  模型函数， model_fn: A model_fn defined by user. Returns `mox.ModelSpec`.
  
  &#160;&#160; optimizer定义， optimizer_fn: An optimizer_fn defined by user. Returns an optimizer.
  
  &#160;&#160; 运行模式选择， run_mode: Only takes mox.ModeKeys.TRAIN or mox.ModeKeys.EVAL or mox.ModeKeys.PREDICT
  
  &#160;&#160; batch大小设置， batch_size: Mini-batch size.
  
 &#160;&#160;  是否自动化batch， auto_batch: If True, an extra dimension of batch_size will be expanded to the first
                     dimension of the return value from `get_split`. Default to True.
                     
  &#160;&#160; 日志以及checkpoint保存位置， log_dir: The directory to save summaries and checkpoints.
  
  &#160;&#160; 最大数量，  max_number_of_steps: Maximum steps for each worker.
                          
  &#160;&#160; 日志打印， log_every_n_steps: Step period to print logs to std I/O.
     
  &#160;&#160; 是否输出模型， export_model: True or False. Where to export model after running the job.



In [46]:
def maybe_download(filename):
    """Download the data from Yann's website, unless it's already here."""
    if not tf.gfile.Exists(DATA_DIRECTORY):
        tf.gfile.MakeDirs(DATA_DIRECTORY)
    filepath = os.path.join(DATA_DIRECTORY, filename)
    if not tf.gfile.Exists(filepath):
        print(filepath," does not exist")
        filepath, _ = urllib.request.urlretrieve(SOURCE_URL + filename, filepath)
        with tf.gfile.GFile(filepath) as f:
            size = f.size()
        print('Successfully downloaded', filename, size, 'bytes.')
    return filepath

# Extract the images
def extract_data(filename, num_images):
    """Extract the images into a 4D tensor [image index, y, x, channels].

    Values are rescaled from [0, 255] down to [-0.5, 0.5].
    """
    print('Extracting', filename)
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(IMAGE_SIZE * IMAGE_SIZE * num_images * NUM_CHANNELS)
        data = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.float32)
        data = (data - (PIXEL_DEPTH / 2.0)) / PIXEL_DEPTH
        data = data.reshape(num_images, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)
        data = numpy.reshape(data, [num_images, -1])
    return data

# Extract the labels
def extract_labels(filename, num_images):
    """Extract the labels into a vector of int64 label IDs."""
    print('Extracting', filename)
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * num_images)
        labels = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.int64)
        num_labels_data = len(labels)
        one_hot_encoding = numpy.zeros((num_labels_data,NUM_LABELS))
        one_hot_encoding[numpy.arange(num_labels_data),labels] = 1
        one_hot_encoding = numpy.reshape(one_hot_encoding, [-1, NUM_LABELS])
    return one_hot_encoding

# Augment training data
def expend_training_data(images, labels):

    expanded_images = []
    expanded_labels = []

    j = 0 # counter
    for x, y in zip(images, labels):
        j = j+1
        if j%100==0:
            print ('expanding data : %03d / %03d' % (j,numpy.size(images,0)))

        # register original data
        expanded_images.append(x)
        expanded_labels.append(y)

        # get a value for the background
        # zero is the expected value, but median() is used to estimate background's value 
        bg_value = numpy.median(x) # this is regarded as background's value        
        image = numpy.reshape(x, (-1, 28))

        for i in range(4):
            # rotate the image with random degree
            angle = numpy.random.randint(-15,15,1)
            new_img = ndimage.rotate(image,angle,reshape=False, cval=bg_value)

            # shift the image with random distance
            shift = numpy.random.randint(-2, 2, 2)
            new_img_ = ndimage.shift(new_img,shift, cval=bg_value)

            # register new training data
            expanded_images.append(numpy.reshape(new_img_, 784))
            expanded_labels.append(y)

    # images and labels are concatenated for random-shuffle at each epoch
    # notice that pair of image and label should not be broken
    expanded_train_total_data = numpy.concatenate((expanded_images, expanded_labels), axis=1)
    numpy.random.shuffle(expanded_train_total_data)

    return expanded_train_total_data

# Prepare MNISt data
def prepare_MNIST_data(use_data_augmentation=True):
    # Get the data.
    train_data_filename = maybe_download('train-images-idx3-ubyte.gz')
    train_labels_filename = maybe_download('train-labels-idx1-ubyte.gz')
    test_data_filename = maybe_download('t10k-images-idx3-ubyte.gz')
    test_labels_filename = maybe_download('t10k-labels-idx1-ubyte.gz')

    # Extract it into numpy arrays.
    train_data = extract_data(train_data_filename, 60000)
    train_labels = extract_labels(train_labels_filename, 60000)
    test_data = extract_data(test_data_filename, 10000)
    test_labels = extract_labels(test_labels_filename, 10000)

    # Generate a validation set.
    validation_data = train_data[:VALIDATION_SIZE, :]
    validation_labels = train_labels[:VALIDATION_SIZE,:]
    train_data = train_data[VALIDATION_SIZE:, :]
    train_labels = train_labels[VALIDATION_SIZE:,:]

    # Concatenate train_data & train_labels for random shuffle
    if use_data_augmentation:
        train_total_data = expend_training_data(train_data, train_labels)
    else:
        train_total_data = numpy.concatenate((train_data, train_labels), axis=1)

    train_size = train_total_data.shape[0]

    return train_total_data, train_size, validation_data, validation_labels, test_data, test_labels


In [47]:
MODEL_DIRECTORY = train_url
LOGS_DIRECTORY = log_url
# Params for Train
training_epochs = 10# 10 for augmented training data, 20 for training data
TRAIN_BATCH_SIZE = 50
display_step = 100
validation_step = 500

# Params for test
TEST_BATCH_SIZE = 5000
import tensorflow.contrib.slim as slim

In [48]:
# Create model of CNN with slim api
def CNN(inputs, is_training=True):
    batch_norm_params = {'is_training': is_training, 'decay': 0.9, 'updates_collections': None}
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                        normalizer_fn=slim.batch_norm,
                        normalizer_params=batch_norm_params):
        x = tf.reshape(inputs, [-1, 28, 28, 1])

        # For slim.conv2d, default argument values are like
        # normalizer_fn = None, normalizer_params = None, <== slim.arg_scope changes these arguments
        # padding='SAME', activation_fn=nn.relu,
        # weights_initializer = initializers.xavier_initializer(),
        # biases_initializer = init_ops.zeros_initializer,
        net = slim.conv2d(x, 32, [5, 5], scope='conv1')
        net = slim.max_pool2d(net, [2, 2], scope='pool1')
        net = slim.conv2d(net, 64, [5, 5], scope='conv2')
        net = slim.max_pool2d(net, [2, 2], scope='pool2')
        net = slim.flatten(net, scope='flatten3')

        # For slim.fully_connected, default argument values are like
        # activation_fn = nn.relu,
        # normalizer_fn = None, normalizer_params = None, <== slim.arg_scope changes these arguments
        # weights_initializer = initializers.xavier_initializer(),
        # biases_initializer = init_ops.zeros_initializer,
        net = slim.fully_connected(net, 1024, scope='fc3')
        net = slim.dropout(net, is_training=is_training, scope='dropout3')  # 0.5 by default
        outputs = slim.fully_connected(net, 10, activation_fn=None, normalizer_fn=None, scope='fco')
    return outputs

下面是训练函数 def train()

In [49]:
def train():

    # Some parameters
    batch_size = TRAIN_BATCH_SIZE
    num_labels = NUM_LABELS

    # Prepare mnist data
    train_total_data, train_size, validation_data, validation_labels, test_data, test_labels = prepare_MNIST_data(True)

    # Boolean for MODE of train or test
    is_training = tf.placeholder(tf.bool, name='MODE')

    # tf Graph input
    x = tf.placeholder(tf.float32, [None, 784])
    y_ = tf.placeholder(tf.float32, [None, 10]) #answer

    # Predict
    y = CNN(x)

    # Get loss of model
    with tf.name_scope("LOSS"):
        loss = slim.losses.softmax_cross_entropy(y,y_)

    # Create a summary to monitor loss tensor
    tf.summary.scalar('loss', loss)

    # Define optimizer
    with tf.name_scope("ADAM"):
        # Optimizer: set up a variable that's incremented once per batch and
        # controls the learning rate decay.
        batch = tf.Variable(0)

        learning_rate = tf.train.exponential_decay(
            1e-4,  # Base learning rate.
            batch * batch_size,  # Current index into the dataset.
            train_size,  # Decay step.
            0.95,  # Decay rate.
            staircase=True)
        # Use simple momentum for the optimization.
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss,global_step=batch)

    # Create a summary to monitor learning_rate tensor
    tf.summary.scalar('learning_rate', learning_rate)

    # Get accuracy of model
    with tf.name_scope("ACC"):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # Create a summary to monitor accuracy tensor
    tf.summary.scalar('acc', accuracy)

    # Merge all summaries into a single op
    merged_summary_op = tf.summary.merge_all()

    # Add ops to save and restore all the variables
    saver = tf.train.Saver()
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer(), feed_dict={is_training: True})

    # Training cycle
    total_batch = int(train_size / batch_size)

    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(LOGS_DIRECTORY, graph=tf.get_default_graph())

    # Save the maximum accuracy value for validation data
    max_acc = 0.

    # Loop for epoch
    for epoch in range(training_epochs):

        # Random shuffling
        numpy.random.shuffle(train_total_data)
        train_data_ = train_total_data[:, :-num_labels]
        train_labels_ = train_total_data[:, -num_labels:]

        # Loop over all batches
        for i in range(total_batch):

            # Compute the offset of the current minibatch in the data.
            offset = (i * batch_size) % (train_size)
            batch_xs = train_data_[offset:(offset + batch_size), :]
            batch_ys = train_labels_[offset:(offset + batch_size), :]

            # Run optimization op (backprop), loss op (to get loss value)
            # and summary nodes
            _, train_accuracy, summary = sess.run([train_step, accuracy, merged_summary_op] , feed_dict={x: batch_xs, y_: batch_ys, is_training: True})

            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)

            # Display logs
            if i % display_step == 0:
                print("Epoch:", '%04d,' % (epoch + 1),
                "batch_index %4d/%4d, training accuracy %.5f" % (i, total_batch, train_accuracy))

            # Get accuracy for validation data
            if i % validation_step == 0:
                # Calculate accuracy
                validation_accuracy = sess.run(accuracy,
                feed_dict={x: validation_data, y_: validation_labels, is_training: False})

                print("Epoch:", '%04d,' % (epoch + 1),
                "batch_index %4d/%4d, validation accuracy %.5f" % (i, total_batch, validation_accuracy))

            # Save the current model if the maximum accuracy is updated
            if validation_accuracy > max_acc:
                max_acc = validation_accuracy
                save_path = saver.save(sess, MODEL_DIRECTORY)
                print("Model updated and saved in file: %s" % save_path)

    print("Optimization Finished!")

    # Restore variables from disk
    saver.restore(sess, MODEL_DIRECTORY)

    # Calculate accuracy for all mnist test images
    test_size = test_labels.shape[0]
    batch_size = TEST_BATCH_SIZE
    total_batch = int(test_size / batch_size)

    acc_buffer = []

    # Loop over all batches
    for i in range(total_batch):
        # Compute the offset of the current minibatch in the data.
        offset = (i * batch_size) % (test_size)
        batch_xs = test_data[offset:(offset + batch_size), :]
        batch_ys = test_labels[offset:(offset + batch_size), :]

        y_final = sess.run(y, feed_dict={x: batch_xs, y_: batch_ys, is_training: False})
        correct_prediction = numpy.equal(numpy.argmax(y_final, 1), numpy.argmax(batch_ys, 1))
        acc_buffer.append(numpy.sum(correct_prediction) / batch_size)

    print("test accuracy for the stored model: %g" % numpy.mean(acc_buffer))


In [50]:
train()

Extracting ./cache/local_data/train-images-idx3-ubyte.gz
Extracting ./cache/local_data/train-labels-idx1-ubyte.gz
Extracting ./cache/local_data/t10k-images-idx3-ubyte.gz
Extracting ./cache/local_data/t10k-labels-idx1-ubyte.gz
expanding data : 100 / 55000
expanding data : 200 / 55000
expanding data : 300 / 55000
expanding data : 400 / 55000
expanding data : 500 / 55000
expanding data : 600 / 55000
expanding data : 700 / 55000
expanding data : 800 / 55000
expanding data : 900 / 55000
expanding data : 1000 / 55000
expanding data : 1100 / 55000
expanding data : 1200 / 55000
expanding data : 1300 / 55000
expanding data : 1400 / 55000
expanding data : 1500 / 55000
expanding data : 1600 / 55000
expanding data : 1700 / 55000
expanding data : 1800 / 55000
expanding data : 1900 / 55000
expanding data : 2000 / 55000
expanding data : 2100 / 55000
expanding data : 2200 / 55000
expanding data : 2300 / 55000
expanding data : 2400 / 55000
expanding data : 2500 / 55000
expanding data : 2600 / 55000
exp

expanding data : 26200 / 55000
expanding data : 26300 / 55000
expanding data : 26400 / 55000
expanding data : 26500 / 55000
expanding data : 26600 / 55000
expanding data : 26700 / 55000
expanding data : 26800 / 55000
expanding data : 26900 / 55000
expanding data : 27000 / 55000
expanding data : 27100 / 55000
expanding data : 27200 / 55000
expanding data : 27300 / 55000
expanding data : 27400 / 55000
expanding data : 27500 / 55000
expanding data : 27600 / 55000
expanding data : 27700 / 55000
expanding data : 27800 / 55000
expanding data : 27900 / 55000
expanding data : 28000 / 55000
expanding data : 28100 / 55000
expanding data : 28200 / 55000
expanding data : 28300 / 55000
expanding data : 28400 / 55000
expanding data : 28500 / 55000
expanding data : 28600 / 55000
expanding data : 28700 / 55000
expanding data : 28800 / 55000
expanding data : 28900 / 55000
expanding data : 29000 / 55000
expanding data : 29100 / 55000
expanding data : 29200 / 55000
expanding data : 29300 / 55000
expandin

expanding data : 52800 / 55000
expanding data : 52900 / 55000
expanding data : 53000 / 55000
expanding data : 53100 / 55000
expanding data : 53200 / 55000
expanding data : 53300 / 55000
expanding data : 53400 / 55000
expanding data : 53500 / 55000
expanding data : 53600 / 55000
expanding data : 53700 / 55000
expanding data : 53800 / 55000
expanding data : 53900 / 55000
expanding data : 54000 / 55000
expanding data : 54100 / 55000
expanding data : 54200 / 55000
expanding data : 54300 / 55000
expanding data : 54400 / 55000
expanding data : 54500 / 55000
expanding data : 54600 / 55000
expanding data : 54700 / 55000
expanding data : 54800 / 55000
expanding data : 54900 / 55000
expanding data : 55000 / 55000


/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch: 0001, batch_index    0/5500, training accuracy 0.20000
Epoch: 0001, batch_index    0/5500, validation accuracy 0.13360
Model updated and saved in file: ./cache/model/
Epoch: 0001, batch_index  100/5500, training accuracy 0.90000
Epoch: 0001, batch_index  200/5500, training accuracy 0.94000
Epoch: 0001, batch_index  300/5500, training accuracy 0.98000
Epoch: 0001, batch_index  400/5500, training accuracy 0.86000
Epoch: 0001, batch_index  500/5500, training accuracy 0.94000
Epoch: 0001, batch_index  500/5500, validation accuracy 0.97120
Model updated and saved in file: ./cache/model/
Epoch: 0001, batch_index  600/5500, training accuracy 0.98000
Epoch: 0001, batch_index  700/5500, training accuracy 0.98000
Epoch: 0001, batch_index  800/5500, training accuracy 1.00000
Epoch: 0001, batch_index  900/5500, training accuracy 0.98000
Epoch: 0001, batch_index 1000/5500, training accuracy 0.96000
Epoch: 0001, batch_index 1000/5500, validation accuracy 0.97960
Model updated and saved in fil

...

Epoch: 0006, batch_index 4800/5500, training accuracy 1.00000
Epoch: 0006, batch_index 4900/5500, training accuracy 1.00000
Epoch: 0006, batch_index 5000/5500, training accuracy 1.00000
Epoch: 0006, batch_index 5000/5500, validation accuracy 0.99460
Epoch: 0006, batch_index 5100/5500, training accuracy 1.00000
Epoch: 0006, batch_index 5200/5500, training accuracy 1.00000
Epoch: 0006, batch_index 5300/5500, training accuracy 1.00000
Epoch: 0006, batch_index 5400/5500, training accuracy 1.00000
Epoch: 0007, batch_index    0/5500, training accuracy 1.00000
Epoch: 0007, batch_index    0/5500, validation accuracy 0.99400
Epoch: 0007, batch_index  100/5500, training accuracy 1.00000
Epoch: 0007, batch_index  200/5500, training accuracy 1.00000
Epoch: 0007, batch_index  300/5500, training accuracy 1.00000
Epoch: 0007, batch_index  400/5500, training accuracy 1.00000
Epoch: 0007, batch_index  500/5500, training accuracy 1.00000
Epoch: 0007, batch_index  500/5500, validation accuracy 0.99480
Ep

Epoch: 0008, batch_index 4800/5500, training accuracy 0.98000
Epoch: 0008, batch_index 4900/5500, training accuracy 1.00000
Epoch: 0008, batch_index 5000/5500, training accuracy 1.00000
Epoch: 0008, batch_index 5000/5500, validation accuracy 0.99480
Epoch: 0008, batch_index 5100/5500, training accuracy 1.00000
Epoch: 0008, batch_index 5200/5500, training accuracy 1.00000
Epoch: 0008, batch_index 5300/5500, training accuracy 1.00000
Epoch: 0008, batch_index 5400/5500, training accuracy 0.98000
Epoch: 0009, batch_index    0/5500, training accuracy 1.00000
Epoch: 0009, batch_index    0/5500, validation accuracy 0.99460
Epoch: 0009, batch_index  100/5500, training accuracy 1.00000
Epoch: 0009, batch_index  200/5500, training accuracy 1.00000
Epoch: 0009, batch_index  300/5500, training accuracy 1.00000
Epoch: 0009, batch_index  400/5500, training accuracy 1.00000
Epoch: 0009, batch_index  500/5500, training accuracy 1.00000
Epoch: 0009, batch_index  500/5500, validation accuracy 0.99480
Ep

Epoch: 0010, batch_index 4600/5500, training accuracy 1.00000
Epoch: 0010, batch_index 4700/5500, training accuracy 1.00000
Epoch: 0010, batch_index 4800/5500, training accuracy 0.98000
Epoch: 0010, batch_index 4900/5500, training accuracy 1.00000
Epoch: 0010, batch_index 5000/5500, training accuracy 1.00000
Epoch: 0010, batch_index 5000/5500, validation accuracy 0.99380
Epoch: 0010, batch_index 5100/5500, training accuracy 1.00000
Epoch: 0010, batch_index 5200/5500, training accuracy 0.98000
Epoch: 0010, batch_index 5300/5500, training accuracy 1.00000
Epoch: 0010, batch_index 5400/5500, training accuracy 1.00000


INFO:tensorflow:Restoring parameters from ./cache/model/


Optimization Finished!
test accuracy for the stored model: 0.9948


以下部分尚未更改

## <a name="predict">2. 预测</a>  



&#8195;&#8195; 在上面训练的基础上，我们可以直接用训练的模型进行预测作业。如读取OBS桶中的数字图片进行识别。input_fn 对输入图片进行简单处理，得到网络允许的输入tensor；model_fn定义一个预测内容，同时，还需定义一个对输出处理的函数output_fn，我们在改函数里对输出进行一个打印输出。
 
  还需在 mox.run()函数中加入如下参数：
  
 &#8195;&#8195; 输出函数 output_fn: A callback with args of results from sess.run.
   
&#8195;&#8195; 模型加载位置 checkpoint_path: Directory or file path of ckpt to restore when `run_mode` is 'evaluation'.
                          Useless when `run_mode` is 'train'.

In [ ]:
####### your coding place： begin###########

#此处必须修改为用户数据存储的OBS位置

# 预测图片在OBS的存储位置。
# eg. 图片名称：  image_number.jpg
#     存储位置为：bucket/test/
src_path = 's3://bucket/test/image_number.jpg'

####### your coding place： end  ###########

In [ ]:
# 可以利用moxing 将需要预测的图片从OBS拷贝到本地
if not mox.file.exists(src_path):
    raise ValueError('Plese verify your src_path!')
dst_path =  './cache/test.jpg'
mox.file.copy(src_path,dst_path)

In [ ]:
image_path = './cache/test.jpg'            # 指定图片位置
checkpoint_url = './cache/log/'         # 指定checkpoint位置，即上一步训练指定的路径的位置。
print(mox.file.exists(image_path))

In [ ]:
import moxing.tensorflow as mox
import os
import tensorflow as tf
from __future__ import print_function
from __future__ import unicode_literals


In [ ]:
def predict(*args):
  def input_fn(run_mode, **kwargs):
    image = tf.read_file(image_path)
    img = tf.image.decode_jpeg(image, channels=1)
    img = tf.image.resize_images(img, [28, 28], 0)
    img = tf.reshape(img, [784])
    return img

  def model_fn(inputs, run_mode, **kwargs):
    x = inputs
    W1 = tf.get_variable(name='W', initializer=tf.zeros([784, 10]))
    b1 = tf.get_variable(name='b', initializer=tf.zeros([10]))
    y = tf.matmul(x, W1) + b1
    predictions = tf.argmax(y, 1)
    return mox.ModelSpec(output_info={'predict': predictions})

  def output_fn(outputs):
    for output in outputs:
      result = output['predict']
      print("The result：",result)

  mox.run(input_fn=input_fn,
          model_fn=model_fn,
          output_fn=output_fn,
          run_mode=mox.ModeKeys.PREDICT,
          batch_size=1,
          auto_batch=False,
          max_number_of_steps=1,
          output_every_n_steps=1,
          checkpoint_path=checkpoint_url)
if __name__ == '__main__':
  tf.app.run(main=predict)

通过预测，我们能够看到结果输出。
